In [ ]:

! pip install tensorflow
! pip install tensorflow-io
! pip install kafka-python
! pip install numpy
! pip install plotly
! pip install pandas
! pip install numpy
! pip install scikit-learn


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from kafka import KafkaConsumer
import plotly.express as px
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio


## Food Delivery Time Prediction Model

Now let’s train a Machine Learning model using an LSTM neural network model for the task of food delivery time prediction:

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (3, 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error', run_eagerly=True)
model.summary()

In [77]:
online_train_ds = tfio.experimental.streaming.KafkaBatchIODataset(
    topics=["result-data"],
    group_id="testzo",
    servers="localhost:19092,localhost:29092,localhost:39092",
    stream_timeout=10000,
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=earliest"
    ],
)


def decode_kafka_online_item(raw_message, raw_key):
    message = tf.io.decode_csv(raw_message, [[0.0] for i in range(3)])
    print("message> ",message)
    key = tf.strings.to_number(raw_key)
    print("key> ",key)
    return (message, key)
  
batch_size = 20
for single_ds in online_train_ds:
    if len(single_ds) >= batch_size:
        single_ds = single_ds.shuffle(buffer_size=batch_size)
        single_ds = single_ds.map(decode_kafka_online_item)
        single_ds = single_ds.batch(batch_size)
    
        model.fit(single_ds, epochs=2)
        tf.keras.models.save_model(model, "./time_prediction_model")
    else:
        print("Not enough data in the dataset. Skipping model fitting.")




2024-01-05 21:46:54.214677: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: session.timeout.ms=7000
2024-01-05 21:46:54.214694: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: max.poll.interval.ms=8000
2024-01-05 21:46:54.214702: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: auto.offset.reset=earliest
2024-01-05 21:46:54.214708: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: group.id=testzo
2024-01-05 21:46:54.214713: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: bootstrap.servers=localhost:19092,localhost:29092,localhost:39092
2024-01-05 21:46:54.214727: I tensorflow_io/core/kernels/kafka_kernels.cc:919] max num of messages per batch: 10000
2024-01-05 21:46:54.214733: I tensorflow_io/core/kernels/kafka_kernels.cc:938] Creating the kafka consumer
2024-01-05 21:46:54.214950: I tensorflow_io/core/kernels/kafka_kernels.cc:945] Subscribing to the kafka topic: result-

message>  [<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>]
key>  Tensor("StringToNumber:0", shape=(), dtype=float32)
Epoch 1/2
500/500 [==============================] - 63s 127ms/step - loss: 57.5211
Epoch 2/2
500/500 [==============================] - 61s 121ms/step - loss: 57.4781
INFO:tensorflow:Assets written to: ./time_prediction_model/assets


INFO:tensorflow:Assets written to: ./time_prediction_model/assets


message>  [<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>]
key>  Tensor("StringToNumber:0", shape=(), dtype=float32)


2024-01-05 21:49:08.426034: E tensorflow_io/core/kernels/kafka_kernels.cc:774] REBALANCE: Local: Revoke partitions
2024-01-05 21:49:08.426878: E tensorflow_io/core/kernels/kafka_kernels.cc:776] Retrieved committed offsets with status code: 0
2024-01-05 21:49:08.426894: I tensorflow_io/core/kernels/kafka_kernels.cc:787] REBALANCE: result-data[0], OFFSET: 20000 ERROR_CODE: 0
2024-01-05 21:49:08.426899: I tensorflow_io/core/kernels/kafka_kernels.cc:806] REBALANCE: Unassigning partitions
2024-01-05 21:49:08.429587: E tensorflow_io/core/kernels/kafka_kernels.cc:774] REBALANCE: Local: Assign partitions
2024-01-05 21:49:08.429996: E tensorflow_io/core/kernels/kafka_kernels.cc:776] Retrieved committed offsets with status code: 0
2024-01-05 21:49:08.430006: I tensorflow_io/core/kernels/kafka_kernels.cc:787] REBALANCE: result-data[0], OFFSET: 20000 ERROR_CODE: 0
2024-01-05 21:49:08.430011: I tensorflow_io/core/kernels/kafka_kernels.cc:802] REBALANCE: Assigning partitions


...

INFO:tensorflow:Assets written to: ./time_prediction_model/assets


Not enough data in the dataset. Skipping model fitting.


2024-01-05 21:57:06.431528: E tensorflow_io/core/kernels/kafka_kernels.cc:774] REBALANCE: Local: Revoke partitions
2024-01-05 21:57:06.432269: E tensorflow_io/core/kernels/kafka_kernels.cc:776] Retrieved committed offsets with status code: 0
2024-01-05 21:57:06.432282: I tensorflow_io/core/kernels/kafka_kernels.cc:787] REBALANCE: result-data[0], OFFSET: 45593 ERROR_CODE: 0
2024-01-05 21:57:06.432288: I tensorflow_io/core/kernels/kafka_kernels.cc:806] REBALANCE: Unassigning partitions
2024-01-05 21:57:06.434846: E tensorflow_io/core/kernels/kafka_kernels.cc:774] REBALANCE: Local: Assign partitions
2024-01-05 21:57:06.435323: E tensorflow_io/core/kernels/kafka_kernels.cc:776] Retrieved committed offsets with status code: 0
2024-01-05 21:57:06.435336: I tensorflow_io/core/kernels/kafka_kernels.cc:787] REBALANCE: result-data[0], OFFSET: 45593 ERROR_CODE: 0
2024-01-05 21:57:06.435341: I tensorflow_io/core/kernels/kafka_kernels.cc:802] REBALANCE: Assigning partitions
2024-01-05 21:57:06.9435

Not enough data in the dataset. Skipping model fitting.


2024-01-05 21:57:16.943623: E tensorflow_io/core/kernels/kafka_kernels.cc:1001] Local: Timed out


In [ ]:
print("Food Delivery Time Prediction")
a = int(input("Age of Delivery Partner: "))
b = float(input("Ratings of Previous Deliveries: "))
c = int(input("Total Distance: "))

features = np.array([[a, b, c]])
print("Predicted Delivery Time in Minutes = ", model.predict(features))